In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important;} *{font-family:'Consolas'; font-size:14;} </style>"))

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
def save_data(df, txt):
    df.to_csv(txt, index=False, encoding="utf-8-sig")
def read_data(txt):
    return pd.read_csv(txt, encoding='utf-8-sig')

# 데이터 읽기

In [4]:
path = 'Data/'

In [5]:
# 공장 데이터: 2*3 list
plant = list()
for i in range(2):
    plant.append(list())
    for j in range(3):
        plant[i].append(read_data(path+'train/'+'plant'+str(i+1)+str(j+1)+'_train_only.csv'))

In [6]:
plant[0][0].columns

Index(['mea_ddhr', 'tem_in_loc', 'hum_in_loc', 'tem_coil_loc', 'tem_out_loc',
       'hum_out_loc', 'cond_loc1', 'cond_24hr', 'cond_48hr', 'tem_in_loc_24hr',
       'tem_in_loc_48hr', 'hum_in_loc_24hr', 'hum_in_loc_48hr',
       'tem_coil_loc_24hr', 'tem_coil_loc_48hr'],
      dtype='object')

In [7]:
# 날짜 데이터: plant1, plant2
date = list()
for i in range(2):
    date.append(read_data(path+'train_data/'+'plant'+str(i+1)+'_date.csv'))

In [8]:
date[0].columns

Index(['mea_ddhr', 'year', 'month', 'day', 'hour', 'week_1', 'week_2',
       'week_3', 'week_4', 'week_5', 'week_6', 'week_7', 'week_8', 'week_9',
       'week_10', 'week_11', 'week_12', 'week_13', 'week_14', 'week_15',
       'week_16', 'week_17', 'week_18', 'week_19', 'week_20', 'week_21',
       'week_22', 'week_23', 'week_24', 'week_25', 'week_26', 'week_27',
       'week_28', 'week_29', 'week_30', 'week_31', 'week_32', 'week_33',
       'week_34', 'week_35', 'week_36', 'week_37', 'week_38', 'week_39',
       'week_40', 'week_41', 'week_42', 'week_43', 'week_44', 'week_45',
       'week_46', 'week_47', 'week_48', 'week_49', 'week_50', 'week_51',
       'week_52'],
      dtype='object')

In [9]:
# 날씨 데이터 - AWS
aws = list()
aws.append(read_data(path+'train_data/'+'new_time_aws_616.csv'))
aws.append(read_data(path+'train_data/'+'new_time_aws_637.csv'))

In [10]:
aws[0].columns

Index(['mea_ddhr', 'aws_loc', 'aws_tem', 'aws_min_rain', 'aws_rain_YN',
       'aws_wind_d', 'aws_wind_s', 'aws_spot_pr', 'aws_sea_pr', 'aws_hum',
       'aws_sun', 'aws_sun_time'],
      dtype='object')

In [11]:
# 날씨 데이터 - asos
asos = read_data(path+'train_data/'+'new_time_asos.csv')

In [12]:
asos.columns

Index(['mea_ddhr', 'asos_tem', 'asos_wind', 'asos_wind_dir', 'asos_hum',
       'asos_steam', 'asos_dew', 'asos_spot_pr', 'asos_sea_pr',
       'asos_tem_grd'],
      dtype='object')

In [13]:
# 날씨 데이터 - 예보 데이터
wt = read_data(path+'train_data/'+'plant_pred.csv')

In [14]:
wt.columns

Index(['mea_ddhr', 'tem_24hr', 'hum_24hr', 'rain_p_24hr', 'wind_s_24hr',
       'wind_d_24hr', 'tem_48hr', 'hum_48hr', 'rain_p_48hr', 'wind_s_48hr',
       'wind_d_48hr', 'sky_1_24hr', 'sky_2_24hr', 'sky_3_24hr', 'sky_4_24hr',
       'sky_1_48hr', 'sky_2_48hr', 'sky_3_48hr', 'sky_4_48hr', 'rain_t_0_24hr',
       'rain_t_1_24hr', 'rain_t_2_24hr', 'rain_t_3_24hr', 'rain_t_0_48hr',
       'rain_t_1_48hr', 'rain_t_2_48hr', 'rain_t_3_48hr'],
      dtype='object')

# 데이터 병합

In [15]:
# 우선은 sample data, plant11(1공장 1location)만 사용
sample = plant[0][0].copy()

In [16]:
# merge on이 되는 시간타입이 다른 부분이 있는지 확인
print(type(sample.mea_ddhr[0]), sample.mea_ddhr[0])
print(type(aws[0].mea_ddhr[0]), aws[0].mea_ddhr[0])
print(type(asos.mea_ddhr[0]), asos.mea_ddhr[0])
print(type(date[0].mea_ddhr[0]), date[0].mea_ddhr[0])
print(type(wt.mea_ddhr[0]), wt.mea_ddhr[0])

<class 'str'> 2016-04-01 00:00:00
<class 'str'> 2016-04-01 00:00:00
<class 'str'> 2016-04-01 03:00:00
<class 'str'> 2016-04-01 00:00:00
<class 'str'> 2016-04-01 00:00:00


우선 시간별로 나누는거부터.. 24시간 후만 뽑아내보자

In [17]:
test = sample.loc[:, ['mea_ddhr', 'cond_24hr', 'tem_in_loc_24hr', 'hum_in_loc_24hr', 'tem_coil_loc_24hr']]

In [18]:
test = pd.merge(test, date[0], how='left', on='mea_ddhr')

In [19]:
aws[0].rename(columns=lambda x: 'dj_'+x, inplace=True)
aws[0].rename({'dj_mea_ddhr':'mea_ddhr'}, axis=1, inplace=True)

In [20]:
aws[1].rename(columns=lambda x: 'sp_'+x, inplace=True)
aws[1].rename({'sp_mea_ddhr':'mea_ddhr'}, axis=1, inplace=True)

In [22]:
for i in range(2):
    test = pd.merge(test, aws[i], how='left', on='mea_ddhr')

In [26]:
test.columns

Index(['mea_ddhr', 'cond_24hr', 'tem_in_loc_24hr', 'hum_in_loc_24hr',
       'tem_coil_loc_24hr', 'year', 'month', 'day', 'hour', 'week_1', 'week_2',
       'week_3', 'week_4', 'week_5', 'week_6', 'week_7', 'week_8', 'week_9',
       'week_10', 'week_11', 'week_12', 'week_13', 'week_14', 'week_15',
       'week_16', 'week_17', 'week_18', 'week_19', 'week_20', 'week_21',
       'week_22', 'week_23', 'week_24', 'week_25', 'week_26', 'week_27',
       'week_28', 'week_29', 'week_30', 'week_31', 'week_32', 'week_33',
       'week_34', 'week_35', 'week_36', 'week_37', 'week_38', 'week_39',
       'week_40', 'week_41', 'week_42', 'week_43', 'week_44', 'week_45',
       'week_46', 'week_47', 'week_48', 'week_49', 'week_50', 'week_51',
       'week_52', 'dj_aws_loc', 'dj_aws_tem', 'dj_aws_min_rain',
       'dj_aws_rain_YN', 'dj_aws_wind_d', 'dj_aws_wind_s', 'dj_aws_spot_pr',
       'dj_aws_sea_pr', 'dj_aws_hum', 'dj_aws_sun', 'dj_aws_sun_time',
       'sp_aws_loc', 'sp_aws_tem', 'sp_aws_min_ra